In [ ]:
import yaml
config = '../inputs/config.yml'

data = yaml.safe_load(open(config))

print(config)
print(data)

In [1]:
import core

config = '../inputs/config.yml'
connection = '../inputs/connection.yml'

schema = 'test'

configuration = core.read_configuration(config)
#print(configuration)
core.create_local_folders_and_files(config)
core.create_database(config, connection)
core.create_extensions(config, connection)
core.create_users(config, connection)
core.create_schemas(config, connection)  # maybe use https://github.com/RazerM/pg_grant to persist the privilege granting?
core.create_policies(config, connection) # maybe use https://github.com/RazerM/pg_grant to persist the privilege granting?
core.grant_subscription_privilege(config, connection) # maybe use https://github.com/RazerM/pg_grant to persist the privilege granting?



INFO: Path '../outputs/airflow_log_local/database/airflow_timescaledb' has been created.
INFO: Path '../outputs/airflow_log_local/database/airflow_timescaledb/01_database' has been created.
INFO: Path '../outputs/airflow_log_local/database/airflow_timescaledb/01_database/01_extensions' has been created.
INFO: Path '../outputs/airflow_log_local/database/airflow_timescaledb/01_database/02_schemas' has been created.
INFO: Path '../outputs/airflow_log_local/database/airflow_timescaledb/01_database/03_functions_and_procedures' has been created.
INFO: Path '../outputs/airflow_log_local/database/airflow_timescaledb/02_roles' has been created.
INFO: Path '../outputs/airflow_log_local/database/airflow_timescaledb/02_roles/access_roles' has been created.
INFO: Path '../outputs/airflow_log_local/database/airflow_timescaledb/02_roles/users' has been created.
INFO: Path '../outputs/airflow_log_local/doc' has been created.
INFO: Path '../outputs/airflow_log_local/log' has been created.
INFO: secret.

In [1]:
import core

# declare the paths to the configuration yaml and
# the connection yaml files.
config = '../inputs/config.yml'
connection = '../inputs/connection.yml'

# drop database, users and roles
core.drop_database(config, connection)
core.drop_users(config, connection)
core.drop_roles(config, connection)

Access granted. Database will be dropped.
INFO: Database airflow_timescaledb dropped.
Access granted. Users will be dropped.
INFO: User alembic has been dropped.
INFO: User replicator has been dropped.
INFO: User analyst has been dropped.
INFO: User gigabeam has been dropped.
Access granted. Database will be dropped.
INFO: Role dropped.
INFO: Role dropped.
INFO: Role dropped.
INFO: Role dropped.
INFO: Role dropped.
INFO: Role dropped.
INFO: Role dropped.
INFO: Role dropped.
INFO: Role dropped.
